# Robustness of the obtained models to Adversarial Examples using the DeepFool algorithm for multi-class problems

## Introduction

This Jupyter notebook was created by Antónia Brito, António Cardoso and Pedro Sousa for the Machine Learning II (CC3043) course at University of Porto. It serves as a practical execution of the _DeepFool_ algorithm as an evaluation strategy for the robustness of the obtained models against adversarial examples.

## Authorship

- **Author:** Antónia Brito, António Cardoso, Pedro Sousa
- **University:** Faculty of Science from University of Porto
- **Course:** Machine Learning II (CC3043)
- **Date:** 05/12/2023


## The *DeepFool* Algorithm

Seyed-Mohsen Moosavi-Dezfooli, Alhussein Fawzi and Pascal Frossard published a paper proposing an algorithm entitled DeepFool to evaluate with a simple and accurate methodology the robustness of a classifier against adversarial examples ([link to the publication in PDF format](https://openaccess.thecvf.com/content_cvpr_2016/papers/Moosavi-Dezfooli_DeepFool_A_Simple_CVPR_2016_paper.pdf)).

For a given classifier and example, the algorithm is set to compute the minimal perturbation that is sufficient to change the estimated label. For the remainder of the section, the following notation will be used:

- $f$: a given classifier that outputs a vector with the probability distribution for the classification associated with its probability index.

- $x$: a given example.

- $X$: the domain of the examples.

- $T$: the domain of test examples available.

- $k$: a possible classification of the considered problem. Thus, the probability of the classification of an example $x$ to be $k$ is $f_k(x)$.

- $\hat{k}(x)$: the estimated classification of a given example. It is noted that $\hat{k}(x) = argmax_k( f_k(x) )$.

- $\hat{r}(x)$: the minimal perturbation for which $\hat{k}(x) \ne \hat{k}(x+\hat{r}(x))$.

The DeepFool algorithm only outputs the value of the minimal perturbation $\hat{r}(x)$ of a given example $x$. The following pseudocode represents the DeepFool algorithm:

![DeepFool Algorithm](./deepfool_alg.png)

Finally, the proposed formal definition for the robustness to adversarial examples of a given classifier is the expected value over the domain of examples for the norm of the minimal perturbation for an example divided by the norm of that same example. For practical purposes, the aforementioned expected value is approximated to the mean value for all examples in the available test domain of the classifier:

$\rho_{adv}(f) = 𝔼_X \frac{||\hat{r}(x)||_2}{||x||_2} ≈ \frac{1}{|T|} ∑_{x\in T} \frac{||\hat{r}(x)||_2}{||x||_2}$


In [1]:
# this notebook was entirely run using Google Colab
# to access the models' data through a cloud drive, this initial code block was needed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Implementation of the DeepFool algorithm for multi-class problems on a given model and input example

In [2]:
# import of relevant libraries

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, regularizers, optimizers
from tensorflow.python.client import device_lib
import pandas as pd
import numpy as np
import pickle
import os
import librosa
from copy import deepcopy

In [3]:
# function that calculates the gradient of the k-th element in the model output, given a input x, with respect to the input x
def get_gradient(model, x, k):

    # compute the k-th value of the model output using x as the input under the watch of tensorflow's GradientTape
    with tf.GradientTape(persistent=True) as tape:
        inputs = [tf.cast(input_value, dtype=tf.float64) for input_value in x]
        for input_value in inputs:
            tape.watch(input_value)
        results = model(inputs)
        results_k = results[0,k]

    # obtain the gradient of the k-th element in the model output w.r.t. the input
    gradients = tape.gradient(results_k, inputs)
    del tape
    return [grad.numpy() for grad in gradients], results


# function that implements the DeepFool algorithm
# model: the model to be used in the algorithm
# x0: the initial input without any perturbation
# eta: an overshoot value to be multiplied with each iteration's perturbation
# max_iter: maximum nummber of iterations  the algorithm is allowed to execute
def deepfool(model, x0, eta=0.01, max_iter=20):

    # obtain the initial estimated label
    f_x0 = model(x0).numpy().flatten()
    label_x0 = f_x0.argsort()[::-1][0]

    loop_i = 0
    xi = deepcopy(x0)
    label_xi = label_x0
    r = []

    # main loop
    while label_xi == label_x0 and loop_i < max_iter:
        w_l = [np.zeros(x_input.shape) for x_input in x0]
        f_l = 0
        fk_wk_min = np.inf
        grad_f_label_x0_on_xi, f_xi = get_gradient(model, xi, label_x0)

        for k in range(10): # k = 0, ..., 9 (possible classes in the problem considered for this project)
            if (k == label_x0):
                continue
            grad_f_k_on_xi, f_xi = get_gradient(model, xi, k)
            w_k = [g_f_k - g_f_label for g_f_k, g_f_label in zip(grad_f_k_on_xi, grad_f_label_x0_on_xi)]
            w_k_norm = np.sqrt(np.sum(np.fromiter([np.linalg.norm(w_k_input)**2 for w_k_input in w_k], dtype=np.float32)))
            f_k = f_xi[0,k] - f_xi[0,label_x0]
            fk_wk = np.linalg.norm(f_k) / (w_k_norm + 1e-3)
            if fk_wk < fk_wk_min:
                w_l, f_l = w_k, f_k
        
        w_l_squared_norm = np.sum(np.fromiter([np.linalg.norm(w_l_input)**2 for w_l_input in w_l], dtype=np.float32))
        f_l_norm = np.linalg.norm(f_l)
        ri_const = f_l_norm / (w_l_squared_norm + 1e-3)
        ri = [ri_const * w_l_input for w_l_input in w_l]
        r.append(ri)
        xi_new = [xi_item + (1+eta)*ri_item for xi_item, ri_item in zip(xi, ri)]
        xi = xi_new
        label_xi = model(xi).numpy().flatten().argsort()[::-1][0]
        loop_i += 1

    # main loop finished
    r_sum = [np.zeros(x_input.shape) for x_input in x0]
    for i in range(len(x0)):
        for r_i in r:
            r_sum[i] += r_i[i][0]

    # return the value of r(x), number of iterations performed, and the new label obtained by adding the perturbation to the input
    return r_sum, loop_i, label_xi


# function to calculate the value of ||r(x)|| / ||x||
def example_robustness(x, r):
    r_norm = np.sqrt(np.sum(np.fromiter([np.linalg.norm(r_input)**2 for r_input in r], dtype=np.float32)))
    x_norm = np.sqrt(np.sum(np.fromiter([np.linalg.norm(x_input)**2 for x_input in x], dtype=np.float32)))
    return r_norm / x_norm


# function to return the robustness of a model
def model_robustness(example_robustness_list):
    mean = np.mean(np.array(example_robustness_list))
    std = np.std(np.array(example_robustness_list))
    return mean, std


### Saving and Loading functions

These functions utilize Python's _Pickle_ library. They allow to save and load the content of any Python variable, in binary, maintaining the structure of its data.

In [4]:
def save_pkl(data, path):
    with open(path, "wb") as saved_data:
        pickle.dump(data, saved_data)
    saved_data.close()

def load_pkl(path):
    to_return = None
    with open(path, "rb") as loaded_data:
        to_return = pickle.load(loaded_data)
    loaded_data.close()
    return to_return

## Methodology for the Empirical Experience using DeepFool

As stated before, the robustness for a classifier is defined, in practice, as the mean value for the norm of the minimal perturbation for an example divided by its norm.


Given that the models constructed in this project were empirically evaluated using a 10-fold cross validation, it is deemed necessary to obtain the robustness for each of the models trained in the cross validation process, both for the CNN and the RNN models.

Hence, at each iteration, the data to be used to test the robustness to adversarial examples of the current model will be the same test data used to compute the cross validation metrics in the notebooks *CNN.ipynb* and *LSTM.ipynb*.

The final results to be present for each of the model architectures will be the mean value and standard deviation of the obtained results for each of the cross validation test datasets.


## Running the DeepFool algorithm on the CNN

In [ ]:
# loading the data to be fed into the CNN model
df_data = load_pkl("/content/drive/MyDrive/Colab Notebooks/urbansound8k_cnn.pkl")

In [ ]:
SAMPLE_RATE = 22050
HOP_LENGTH = round(SAMPLE_RATE * 0.0125)
TIME_SIZE = 4*SAMPLE_RATE//HOP_LENGTH+1

for f in range(1, 10+1):

    # this list will hold the values ||r(x)|| / ||x|| initially mentioned on the notebook 
    robustness_values_cnn_fold = []

    # utilize only the data of the f-th fold
    X_fold = df_data[df_data['fold'] == f"fold{f}"]

    # separate the data for each input stream of the CNN model (mel-scaled spectrograms, chromagrams, 1D features - stacked)
    # explicit casts to assure type integrity of the data before being passed to the model
    X_mel = np.asarray(X_fold["mel_spec"].to_list()).astype(np.float32)
    X_chroma = np.asarray(X_fold["chroma"].to_list()).astype(np.float32)
    centroid = np.asarray(tuple(X_fold["spectral_centroid"].to_list())).astype(np.float32)
    bandwidth = np.asarray(tuple(X_fold["spectral_bandwidth"].to_list())).astype(np.float32)
    flatness = np.asarray(tuple(X_fold["spectral_flatness"].to_list())).astype(np.float32)
    rolloff = np.asarray(tuple(X_fold["spectral_rolloff"].to_list())).astype(np.float32)
    X_1d = np.stack([centroid,bandwidth,flatness,rolloff], axis=-1)
    X_1d = X_1d.reshape(-1, TIME_SIZE, 4)

    # load the model from the performance evaluation cross validation run that used the f-th fold as the test fold
    # the compile keyword argument was set to false such that the model can be explicitly re-compiled using the compile settings defined on CNN.ipynb
    fold_model_cnn = keras.models.load_model(f"/content/drive/MyDrive/Colab Notebooks/CNN Models/cnn_model{f}.h5", compile=False)
    fold_model_cnn.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    # begin the run of each example in the f-th fold and the corresponding model on the DeepFool algorithm
    for i in range(len(X_fold)):

        # prepare the input for the implemented DeepFool function
        example_input = [np.array([X_mel[i]]), np.array([X_chroma[i]]), np.array([X_1d[i]])]

        # run DeepFool
        perturbation, iters, fool_label = deepfool(fold_model_cnn, example_input)

        # save the ||r(x)|| / ||x|| value into the list defined in the beginning of the "fold" for loop
        robustness_values_cnn_fold.append(example_robustness(example_input, perturbation))

    # to save the results of each fold
    save_pkl(robustness_values_cnn_fold, f"/content/drive/MyDrive/Colab Notebooks/robustness_values_cnn_fold{f}.pkl")


In [ ]:
# check each folds' models results
for f in range(1, 10+1):
    robustness_values_cnn_fold = load_pkl(f"/content/drive/MyDrive/Colab Notebooks/robustness_values_cnn_fold{f}.pkl")
    mean_robustness_cnn, std_robustness_cnn = model_robustness(robustness_values_cnn_fold)
    print(f"Fold {f} - The CNN model has a robustness of {mean_robustness_cnn: .7f} +/- {std_robustness_cnn: .7f}.")

Fold 1 - The CNN model has a robustness of  0.0000998 +/-  0.0001057.
Fold 2 - The CNN model has a robustness of  0.0000744 +/-  0.0000929.
Fold 3 - The CNN model has a robustness of  0.0001274 +/-  0.0001670.
Fold 4 - The CNN model has a robustness of  0.0000907 +/-  0.0001367.
Fold 5 - The CNN model has a robustness of  0.0001199 +/-  0.0001449.
Fold 6 - The CNN model has a robustness of  0.0000607 +/-  0.0000818.
Fold 7 - The CNN model has a robustness of  0.0000913 +/-  0.0000947.
Fold 8 - The CNN model has a robustness of  0.0001069 +/-  0.0001206.
Fold 9 - The CNN model has a robustness of  0.0001230 +/-  0.0001500.
Fold 10 - The CNN model has a robustness of  0.0000922 +/-  0.0000921.


The results for the various folds' models indicate that they are not very robust to adversarial examples, as the norm of the minimal perturbation to alter the models' predictions is very small relatively to the corresponding input's norm (about 0.00607% to 0.01274% of the original input's norm).

## Running the DeepFool algorithm for the RNN

In [5]:
# loading the data to be fed into the RNN model
df_data = load_pkl("/content/drive/MyDrive/Colab Notebooks/urbansound8k_rnn.pkl")

In [ ]:

for f in range(1, 10+1):

    # this list will hold the values ||r(x)|| / ||x|| initially mentioned on the notebook 
    robustness_values_rnn_fold = []

    # utilize only the data of the f-th fold
    X_fold = df_data[df_data['fold'] == f"fold{f}"]

    # the data to be inputed into the RNN model (spectogram)
    # explicit cast to assure type integrity of the data before being passed to the model
    X_spec = np.asarray(X_fold["spec"].to_list()).astype(np.float32)

    # load the model from the performance evaluation cross validation run that used the f-th fold as the test fold
    # the compile keyword argument was set to false such that the model can be explicitly re-compiled using the compile settings defined on LSTM.ipynb
    fold_model_rnn = keras.models.load_model(f"/content/drive/MyDrive/Colab Notebooks/RNN Models/rnn_model{f}.h5", compile=False)
    fold_model_rnn.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    # begin the run of each example in the f-th fold and the corresponding model on the DeepFool algorithm
    for i in range(len(X_fold)):

        # prepare the input for the implemented DeepFool function
        example_input = [np.array([X_spec[i]])]

        # run DeepFool
        # eta keyword argument was set to 10^6 due to the extremely small order of magnitude of each iteration's perturbation (between 10^-8 and 10^-5)
        perturbation, iters, fool_label = deepfool(fold_model_rnn, example_input, eta=1e6)

        # save the ||r(x)|| / ||x|| value into the list defined in the beginning of the "fold" for loop
        robustness_values_rnn_fold.append(example_robustness(example_input, perturbation))

    # to save the results of each fold
    save_pkl(robustness_values_rnn_fold, f"/content/drive/MyDrive/Colab Notebooks/robustness_values_rnn_fold{f}.pkl")


In [8]:
# check each folds' models results
for f in range(1, 10+1):
    robustness_values_rnn_fold = load_pkl(f"/content/drive/MyDrive/Colab Notebooks/robustness_values_rnn_fold{f}.pkl")
    mean_robustness_rnn, std_robustness_rnn = model_robustness(robustness_values_rnn_fold)
    print(f"Fold {f} - The RNN model has a robustness of {mean_robustness_rnn: .7f} +/- {std_robustness_rnn: .7f}.")

Fold 1 - The RNN model has a robustness of  0.0009903 +/-  0.0012117.
Fold 2 - The RNN model has a robustness of  0.0010233 +/-  0.0011046.
Fold 3 - The RNN model has a robustness of  0.0015006 +/-  0.0011319.
Fold 4 - The RNN model has a robustness of  0.0012471 +/-  0.0010433.
Fold 5 - The RNN model has a robustness of  0.0015296 +/-  0.0013027.
Fold 6 - The RNN model has a robustness of  0.0007348 +/-  0.0010656.
Fold 7 - The RNN model has a robustness of  0.0011942 +/-  0.0010320.
Fold 8 - The RNN model has a robustness of  0.0013856 +/-  0.0013305.
Fold 9 - The RNN model has a robustness of  0.0009796 +/-  0.0011695.
Fold 10 - The RNN model has a robustness of  0.0008553 +/-  0.0008744.


The RNN results show some improvement compared to the CNN ones by a relative factor of approximately 10 times higher.

However, the results show that the model robustness is considerably weak, as the magnitude of the minimal perturbation that changes the model prediction is quite small, ranging from 0.099% to 0.152% of the original input's magnitude.

## Conclusion

The DeepFool algorithm has proven its intended utility case in this notenook, by allowing to evaluate how the composed models for this project react to adversarial examples and finding metrics for the robustness of these models in regard to their predictions stability.

Unfortunately, the models presented a not-so-rich performance given the results computed above. However, one course of action to improve the robustness of these examples would be to use DeepFool as an auxiliar method to fine-tune the models' parameters such that their performances against adversarial inputs are more concise and stable.

This experiment has widened the need to design and create models that perform better against perturbations, using the DeepFool algorithm as one of the benchmark algorithms to obtain metrics.

## References

- Moosavi-Dezfooli, S.-M., Fawzi, A., Frossard, P., Polytechnique, E. and De Lausanne, F. (2016). DeepFool: a simple and accurate method to fool deep neural networks. [online] Available at: https://openaccess.thecvf.com/content_cvpr_2016/papers/Moosavi-Dezfooli_DeepFool_A_Simple_CVPR_2016_paper.pdf [Accessed 3 Dec. 2023].

- Morgan, A. (2022). A Review of DeepFool: a simple and accurate method to fool deep neural networks. [online] Machine Intelligence and Deep Learning. Available at: https://medium.com/machine-intelligence-and-deep-learning-lab/a-review-of-deepfool-a-simple-and-accurate-method-to-fool-deep-neural-networks-b016fba9e48e#:~:text=DeepFool%20finds%20the%20minimal%20perturbations [Accessed 3 Dec. 2023].

- TensorFlow. (2023). Introduction to gradients and automatic differentiation | TensorFlow Core. [online] Available at: https://www.tensorflow.org/guide/autodiff.